## 1. Intuition

### 1.0 What is Naive Bayes Algorithm?

Naive Bayes is a supervised classification algorithm which belongs to the family of "Probabilistic Classifiers". As the name suggests, it is uses Bayes' theorem at its core. You can get a great intuitive sense about conditional probability from lectures of Prof John Tsitsiklis. 

`"You know know something about the world, and based on what you know, you setup a probability model and you write down probabilities about the different outcomes. Then someone gives you some new information, which changes your believes and thus changes the probabilities of your outcomes."`

Let's say you have a sample space the probabilities for different events are given as below - 

<img src="img.jpg" width='500'>

Now let's say someone tells you that B has occured. Now, the area outside the circle that represents B is meaningless since the new sample space is now B. This means, we need to revise the probabilities that have been assiged to the spaces inside B. 

Now someone asks you, what is the probability that A occurs, given that B has occured. Since B has occured and the new sample space is now B, the area that was represented by $P(A \cap B)$ is now the $P(A \mid B)$.

What is the probability of this new section? Well, we can just use simple ratios for solving this.

$$ P (A \mid B) = {P(A \cap B) \over P(B)} = {0.3 \over (0.3+0.2)}$$

Now, the whole excersice we did above can also be done for when A has occured. Meaning - 

$$ P (B \mid A) = {P(A \cap B) \over P(A)}$$

Using the 2, we can write the famous equation - 

$$ P (A \mid B) = {P (B \mid A) \times P(A) \over P(B)}$$

## 2. Implementation

## 3. Derivation

## 4. Development

## 5. References

- https://www.youtube.com/watch?v=j9WZyLZCBzs&list=PLUl4u3cNGP61MdtwGTqZA0MreSaDybji8 (For Probability theory) 